In [20]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 7.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 4.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import csv
import warnings
warnings.filterwarnings("ignore")
csv.field_size_limit(1000000000)

1000000000

In [3]:
df = pd.read_csv("state-of-the-union.csv")
df.columns=['year_of_the_speech','text_of_the_speech']

In [4]:
df

,year_of_the_speech,text_of_the_speech
0,1790,\nState of the Union Address\nGeorge Washingto...
1,1791,\nState of the Union Address\nGeorge Washingto...
2,1792,\nState of the Union Address\nGeorge Washingto...
3,1793,\nState of the Union Address\nGeorge Washingto...
4,1794,\nState of the Union Address\nGeorge Washingto...
...,...,...
220,2008,\nState of the Union Address\nGeorge W. Bush\n...
221,2009,\nAddress to Joint Session of Congress \nBarac...
222,2010,\nState of the Union Address\nBarack Obama \nJ...
223,2011,\nState of the Union Address\nBarack Obama \nJ...


In [5]:
sum(df["text_of_the_speech"].isna())

0

In [6]:
stopwords_from_git = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can't", "cannot", "could", "couldn't", "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn't", "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "isn't", "it", "it's", "its", "itself", "let's", "me", "more", "most", "mustn't", "my", "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "shan't", "she", "she'd", "she'll", "she's", "should", "shouldn't", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "won't", "would", "wouldn't", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]

In [7]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
stopwords.extend(stopwords_from_git)
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
import gensim
import re
import pickle
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(t) for t in tokens]

def stemming(tokens):
    stemmer = SnowballStemmer("english")
    return [stemmer.stem(t) for t in tokens]

In [35]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
def preprocess(text):
    
    # tokens
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stopwords and t.isalpha()]
    tokens = [t for t in tokens if len(t) > 1] # removing stopwords and punctuations
    
    # lammetize
    lemmatized_tokens = lemmatize(tokens)
    
    # stemming
    stemmed_tokens = stemming(lemmatized_tokens)
    
    return stemmed_tokens

In [11]:
df["processed_speech"] = df.text_of_the_speech.apply(preprocess)

In [12]:
df

,year_of_the_speech,text_of_the_speech,processed_speech
0,1790,\nState of the Union Address\nGeorge Washingto...,"[state, union, address, georg, washington, dec..."
1,1791,\nState of the Union Address\nGeorge Washingto...,"[state, union, address, georg, washington, oct..."
2,1792,\nState of the Union Address\nGeorge Washingto...,"[state, union, address, georg, washington, nov..."
3,1793,\nState of the Union Address\nGeorge Washingto...,"[state, union, address, georg, washington, dec..."
4,1794,\nState of the Union Address\nGeorge Washingto...,"[state, union, address, georg, washington, nov..."
...,...,...,...
220,2008,\nState of the Union Address\nGeorge W. Bush\n...,"[state, union, address, georg, bush, januari, ..."
221,2009,\nAddress to Joint Session of Congress \nBarac...,"[address, joint, session, congress, barack, ob..."
222,2010,\nState of the Union Address\nBarack Obama \nJ...,"[state, union, address, barack, obama, januari..."
223,2011,\nState of the Union Address\nBarack Obama \nJ...,"[state, union, address, barack, obama, januari..."


# tf-idf weighte ddocument vectors

In [19]:
import gensim
from gensim import corpora


dictionary = corpora.Dictionary(df["processed_speech"])

# 2nd iteration
bag_of_words = [dictionary.doc2bow(item) for item in df["processed_speech"]]

# tf-idf scores
tf_idf = gensim.models.TfidfModel(bag_of_words)
tf_idf_corpus = tf_idf[bag_of_words]

tf_idf_corpus[0][0]

(0, 0.01120939809115203)

# LSI

In [27]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# topic range 5-30
start_topics = 5
end_topics = 30
step_size = 5
num_topics_list = list(range(start_topics, end_topics + 1, step_size))

best_coherence_value = float('-inf')
best_lsi_model = None

coherence_scores = []

for num_topics in num_topics_list:
    # training
    lsi_model = LsiModel(tf_idf_corpus, id2word=dictionary, num_topics=num_topics, chunksize=100, decay=0.5, random_seed=24)
    
    # coherence score
    coherence_model = CoherenceModel(model=lsi_model, texts=df["processed_speech"], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    coherence_scores.append(coherence_score)
    
    # save the model
    if coherence_score > best_coherence_value:
        best_coherence_value = coherence_score
        best_lsi_model = lsi_model

# optimal number of topics
best_num_topics = num_topics_list[coherence_scores.index(max(coherence_scores))]

print("Coherence scores for each number of topics:", coherence_scores)
print("Best number of topics:", best_num_topics)


Coherence scores for each number of topics: [0.541425728334292, 0.41568350780420094, 0.3914265843470043, 0.3883674108732783, 0.3796490520250867, 0.38540985279640216]
Best number of topics: 5


In [35]:
lsi_topics = best_lsi_model.print_topics(num_topics=5, num_words=20)
lsi_topics

[(0,
  '0.209*"tonight" + 0.186*"job" + 0.157*"child" + 0.147*"help" + 0.130*"program" + 0.129*"school" + 0.123*"budget" + 0.120*"america" + 0.115*"terrorist" + 0.114*"cut" + 0.109*"iraq" + 0.104*"let" + 0.095*"spend" + 0.095*"colleg" + 0.091*"challeng" + 0.088*"get" + 0.084*"know" + 0.083*"today" + 0.082*"tax" + 0.081*"deficit"'),
 (1,
  '-0.294*"terrorist" + -0.230*"iraq" + -0.200*"iraqi" + -0.132*"terror" + -0.122*"tonight" + -0.110*"al" + 0.103*"program" + -0.094*"afghanistan" + -0.087*"qaeda" + -0.085*"regim" + -0.073*"saddam" + -0.071*"child" + 0.071*"econom" + 0.071*"agricultur" + 0.068*"farm" + 0.066*"interst" + -0.062*"hussein" + 0.062*"problem" + 0.062*"corpor" + -0.060*"america"'),
 (2,
  '0.319*"terrorist" + 0.220*"iraq" + 0.209*"iraqi" + -0.151*"job" + 0.145*"terror" + -0.126*"colleg" + -0.119*"cut" + 0.110*"regim" + 0.108*"al" + 0.103*"afghanistan" + -0.101*"child" + -0.095*"school" + -0.093*"parent" + -0.089*"get" + 0.084*"enemi" + -0.079*"challeng" + -0.079*"deficit" + 

In [37]:
# we will use 10 since we need 10 and it has the 2nd best coherence value 
target_lsi_model = LsiModel(tf_idf_corpus, 
                            id2word=dictionary, 
                            num_topics=10, 
                            chunksize=100,  
                            decay=0.5, 
                            random_seed=24)

# topics
lsi_topics = target_lsi_model.print_topics(num_topics=10, num_words=20)
lsi_topics

[(0,
  '0.209*"tonight" + 0.185*"job" + 0.157*"child" + 0.147*"help" + 0.129*"school" + 0.129*"program" + 0.123*"budget" + 0.121*"america" + 0.115*"terrorist" + 0.114*"cut" + 0.108*"iraq" + 0.104*"let" + 0.096*"spend" + 0.094*"colleg" + 0.091*"challeng" + 0.088*"get" + 0.084*"know" + 0.083*"today" + 0.082*"tax" + 0.081*"deficit"'),
 (1,
  '-0.290*"terrorist" + -0.227*"iraq" + -0.196*"iraqi" + -0.131*"terror" + -0.121*"tonight" + -0.109*"al" + 0.107*"program" + -0.094*"afghanistan" + -0.085*"regim" + -0.083*"qaeda" + -0.077*"saddam" + -0.072*"child" + 0.071*"econom" + 0.070*"agricultur" + 0.069*"farm" + 0.068*"interst" + -0.067*"hussein" + 0.063*"corpor" + 0.062*"problem" + -0.059*"america"'),
 (2,
  '-0.323*"terrorist" + -0.223*"iraq" + -0.212*"iraqi" + 0.151*"job" + -0.147*"terror" + 0.125*"colleg" + 0.118*"cut" + -0.111*"regim" + -0.109*"al" + -0.104*"afghanistan" + 0.100*"child" + 0.094*"school" + 0.093*"parent" + 0.089*"get" + -0.085*"enemi" + 0.080*"deficit" + 0.079*"let" + 0.078*

In [39]:
lsi_topics[1]

(1,
 '-0.290*"terrorist" + -0.227*"iraq" + -0.196*"iraqi" + -0.131*"terror" + -0.121*"tonight" + -0.109*"al" + 0.107*"program" + -0.094*"afghanistan" + -0.085*"regim" + -0.083*"qaeda" + -0.077*"saddam" + -0.072*"child" + 0.071*"econom" + 0.070*"agricultur" + 0.069*"farm" + 0.068*"interst" + -0.067*"hussein" + 0.063*"corpor" + 0.062*"problem" + -0.059*"america"')

# LDA